In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from IPython.display import Image
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

import selenium
from selenium import webdriver
import csv
import time
import urllib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException,ElementNotVisibleException,TimeoutException,ElementClickInterceptedException

In [2]:
def scrap_carsdekho(web_link):
    driver=webdriver.Chrome("chromedriver") ##Opening web driver
    driver.get(web_link) ## Opeing web link
    time.sleep(1)
    location_selection_link=driver.find_element_by_xpath("//li[@data-slug='/usedCars']/a").get_attribute('href')
    driver.get(location_selection_link)
    time.sleep(1)
    car_links=[]
    for number,location in enumerate(driver.find_elements_by_xpath("//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li")):
#         print(number)
        time.sleep(2)
        driver.find_element_by_xpath(f"//ul[@class='listing gsc_col-md-9 gsc_col-lg-9']/li[{number+1}]").click()
        n=0
        i=900
        while n!=10:
            driver.execute_script("window.scrollBy(0, arguments[0]);", i)
            time.sleep(5)
            n+=1
            i+=250
        for link in driver.find_elements_by_xpath("//div[@class='gsc_col-xs-7 carsName']"):
            car_links.append(link.find_element_by_xpath('a').get_attribute('href')) # each car link
        driver.get(location_selection_link)
    print(len(car_links))
    values={
         'Car_name':[],
         'Price' : [],
         'Kilometers': [],
         'Year of Purchase': [],
         'Owner': [],
         'Fuel': [],
         'Transmission': [],
         'RTO': [],
#          'Insurance': [],
         'Insurance Type': [],
         'Feature' : [],
         'Safety' : [],
         'Interior' : [],
         'Exterior' : []
           }
    
    for link in car_links:
        try:
            driver.get(link)
            engin_ext_int=[]
            values['Car_name'].append(driver.find_element_by_xpath("//div[@class='gsc_col-xs-12']/h1").text)
            values['Price'].append(driver.find_element_by_xpath("//div[@class='priceSection']").text)
            values["Insurance Type"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[9]").text.split('\n')[1])
            values["RTO"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[7]").text.split('\n')[1])
            values["Transmission"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[8]").text.split('\n')[1])
            values["Fuel"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[3]").text.split('\n')[1])
            values["Owner"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[6]").text.split('\n')[1])
            values["Year of Purchase"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[1]").text.split('\n')[1])
            values["Kilometers"].append(driver.find_element_by_xpath("//ul[@class='gsc_row detailsList']/li[4]").text.split('\n')[1])
            
            plus_buttons=driver.find_elements_by_css_selector("div.content  ")
            for button in plus_buttons:
                driver.execute_script("arguments[0].setAttribute('class','content  on')", button) ## Opening all plus buttons
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[1]/div")
                values['Feature'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[1]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Feature'].append("-")
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[2]/div")
                values['Safety'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[2]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Safety'].append('-')
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[3]/div")
                values['Interior'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[3]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Interior'].append('-')
            try:
                driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[4]/div")
                values['Exterior'].append(', '.join(driver.find_element_by_xpath("//div[@class='accordion UCFeatureBox']/div[4]/div").text.split("\n")))
            except NoSuchElementException as e:
                values['Exterior'].append('-')
        except NoSuchElementException as e:
            continue
    return values
data=scrap_carsdekho('https://www.cardekho.com/')
pd.DataFrame(data).to_csv("cardekho.csv")
pd.DataFrame(data)

1756


,Car_name,Price,Kilometers,Year of Purchase,Owner,Fuel,Transmission,RTO,Insurance Type,Feature,Safety,Interior,Exterior
0,2011 Honda City,"₹ 3,99,500","38,647 Kms",2011,1st Owner,Petrol,Manual,GJ01,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Brake Assist, Center...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
1,2019 Maruti Swift Dzire,"₹ 6,65,000","9,718 Kms",2019,1st Owner,Petrol,Manual,GJ09,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Brake Assist, Center...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter..."
2,2012 Maruti Alto K10,"₹ 1,77,500","89,579 Kms",2012,1st Owner,Petrol,Manual,GJ01,Third Party insurance,"Power Steering, Remote Trunk Opener, Remote Fu...","Halogen Headlamps, Rear Seat Belts, Seat Belt ...","Air Conditioner, Heater, Electronic Multi Trip...","Adjustable Head Lights, Manually Adjustable Ex..."
3,2016 Maruti Wagon R,"₹ 3,60,000","81,674 Kms",2016,1st Owner,Petrol,Manual,GJ18,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter..."
4,2020 Maruti Eeco,"₹ 4,72,000","17,116 Kms",2020,1st Owner,Petrol,Manual,GJ01,Third Party insurance,"Low Fuel Warning Light, Accessory Power Outlet...","Anti Lock Braking System, Child Safety Locks, ...","Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,2016 Toyota Innova Crysta,"₹ 17,40,000","70,370 Kms",2016,1st Owner,Diesel,Manual,MH12,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Brake Assist, Center...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ..."
1251,2015 Hyundai Verna,"₹ 5,71,000","89,087 Kms",2014,1st Owner,Diesel,Automatic,MH08,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
1252,2014 Honda City,"₹ 6,44,836","33,378 Kms",2014,2nd Owner,Petrol,Manual,MH09,Third Party insurance,"Power Steering, Power Windows Front, Power Win...","Anti Lock Braking System, Centeral Locking, Po...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe..."
1253,2015 Maruti Swift,"₹ 5,38,000","57,664 Kms",2015,1st Owner,Diesel,Manual,MH12,Third Party insurance,"Foldable Rear Seat, Power Steering, Power Wind...","Anti Lock Braking System, Brake Assist, Center...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ..."


In [95]:
driver=webdriver.Chrome('chromedriver')
driver.get('https://www.cars24.com/buy-used-cars/')
time.sleep(1)
# driver.find_element_by_xpath('//div[@class="_1K8Qe"]/button').click()


In [90]:
# car_loc=[]
# for i in driver.find_elements_by_xpath('//ul[@class="_16Bvy"]/li'):
#     car_loc.append(i.text)
# car_loc

['New Delhi',
 'Noida',
 'Gurgaon',
 'Mumbai',
 'Pune',
 'Bengaluru',
 'Hyderabad',
 'Chennai',
 'Kolkata',
 '']

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=94.0.4606.61)
